In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Rashmitha\\OneDrive\\Documents\\Self Learning\\End-to-end-Machine-Learning-project-using-MLFlow\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Rashmitha\\OneDrive\\Documents\\Self Learning\\End-to-end-Machine-Learning-project-using-MLFlow'

In [29]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [30]:
pip install python-box


Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install ensure

Note: you may need to restart the kernel to use updated packages.


In [31]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml,create_directories

In [32]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [33]:
import os
import urllib.request as request
import zipfile
from src.mlProject import logger
from src.mlProject.utils.common import get_size

In [34]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")


    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
  

In [38]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-12-03 17:50:47,601: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-03 17:50:47,602: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-03 17:50:47,603: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-12-03 17:50:47,604: INFO: common: created directory at: artifacts]
[2023-12-03 17:50:47,605: INFO: common: created directory at: artifacts/data_ingestion]


[2023-12-03 17:50:48,114: INFO: 1345919267: artifacts/data_ingestion/data.zip download! with following info: 
Server: GitHub.com
Date: Sun, 03 Dec 2023 12:20:48 GMT
Content-Type: text/html; charset=utf-8
Vary: X-PJAX, X-PJAX-Container, Turbo-Visit, Turbo-Frame, Accept-Encoding, Accept, X-Requested-With
ETag: W/"e96422b21fc2c1182e315df9986972ad"
Cache-Control: max-age=0, private, must-revalidate
Strict-Transport-Security: max-age=31536000; includeSubdomains; preload
X-Frame-Options: deny
X-Content-Type-Options: nosniff
X-XSS-Protection: 0
Referrer-Policy: no-referrer-when-downgrade
Content-Security-Policy: default-src 'none'; base-uri 'self'; child-src github.com/assets-cdn/worker/ gist.github.com/assets-cdn/worker/; connect-src 'self' uploads.github.com www.githubstatus.com collector.github.com raw.githubusercontent.com api.githubcopilot.com api.github.com github-cloud.s3.amazonaws.com github-production-repository-file-5c1aeb.s3.amazonaws.com github-production-upload-manifest-file-7fdc

BadZipFile: File is not a zip file